<p><img src="https://files.cdn.thinkific.com/file_uploads/133292/images/a9e/c1a/bc3/P02.jpg"></p>

`MCSDSC02P01V3______`

`MCSDSC02P01V2______`

`MCSDSELIMC02P01V1010621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Замечаний нет, проект принят!
Ниже добавил несколько комментариев.

## Постановка задачи

***Нам предстоит выяснить***:
* На какие дни и на какие рестораны, подключенные к нашему приложению, чаще приходят заказы на резерв стола
* В какое время чаще бронируют столы через наше приложение
* На какое количество человек чаще приходит резерв (будем смотреть по отрезкам времени)

## Импорт данных и библиотек

In [25]:
import pandas as pd
import numpy as np
import pathlib as pl

home_path = pathlib.Path.home()
data_path = Path(home_path, 'mcs_ds_02/data', 'bez_ocheredi.csv')
bo_data = pd.read_csv(data_path)

Прочитаем файл `bez_ocheredi.csv` и сохраним его в переменную `bo_data`.

In [59]:
bo_data.head(10)

,userID,restaraunt,weekday,ordered,guests
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
5,51e0962387f7,Там Ем,Friday,12:30,2
6,c4f260a2b48a,NaN,Saturday,15:00,6
7,6b71309d6a8a,Крафтейнер,Monday,10:30,1
8,c5b969cb63a2,Бар Сибирь,Wednesday,15:30,2
9,b1570962e68c,Бар Сибирь,Wednesday,12:30,3


Получим первые 10 строк таблицы.

In [29]:
bo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userID        1024 non-null   object
 1    ресторан     959 non-null    object
 2    weekday      1024 non-null   object
 3    ordered      1024 non-null   object
 4    Столик для   1024 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 40.1+ KB


Index(['userID ', ' ресторан ', ' weekday', ' ordered ', ' Столик для '], dtype='object')

Получим общую информацию о данных из таблицы.

In [30]:
bo_data.columns

Index(['userID ', ' ресторан ', ' weekday', ' ordered ', ' Столик для '], dtype='object')

Столбцы в `bo_data` содержат следующую информацию:

* userID - уникальный номер пользователя
* ресторан - название ресторана
* weekday - день недели бронирования стола
* ordered - время бронирования
* Столик для - количество гостей на резерв


### Выводы

С помощью модуля *pathlib* и библиотеки *pandas* мы прочитали dataset, выяснили какие типы данных хранятся в каждом столбце, а так же название столбцов. Следующим шагом будет предобработка данных для работы с ними

## Предобработка данных

Получим названия столбцов.

In [298]:
columns_name = bo_data.columns

Обращаем внимание, что помимо названий на разном языке, в них есть пробелы, которые затрудняют доступ к данным. Переименуем столбцы и проверим результат.

In [40]:
bo_data.rename(columns = {bo_data.columns[0]:'userID', bo_data.columns[1]:'restaraunt',
                          bo_data.columns[2]:'weekday', bo_data.columns[3]:'ordered',
                         bo_data.columns[4]:'guests'}, inplace=True)

<h3><font color="#89DD00">Комментарий ассистента</font></h3>  
  
Интересный вариант решения, вот как еще можно было решить:  
```bo_data.set_axis(['userID', 'restaurant', 'weekday', 'order_time', 'people_quantity'], axis='columns', inplace=True)```

In [300]:
columns_sum = ''
for i in range(len(columns_name)):
    columns_sum += columns_name[i]
print(columns_sum)


userIDrestarauntweekdayorderedguests


Еще раз проверим данные на наличие пропусков, просуммировав их.

In [67]:
bo_data['restaraunt'].fillna('admin', inplace=True)

Менеджер проекта предупреждал, что некоторые заказы заводились на тестовую карточку кафе без названия, поэтому есть смысл заполнить такой пропуск словом admin.

In [68]:
bo_data.isna().sum()

userID        0
restaraunt    0
weekday       0
ordered       0
guests        0
dtype: int64

Проверяем, что пропусков не осталось.

In [83]:
bo_data.duplicated().sum()

0

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [82]:
bo_data.drop_duplicates(inplace=True)

In [304]:
bo_data.reset_index(drop=True, inplace=True)

Скорее всего, такие полные дубликаты говорят о сбое при выгрузке данных. Необходимо сообщить об этом.

Еще одна проблема, о которой предупреждал менеджер проекта — дубликаты карточки Iskra Bar с неправильными названием бара - "Искра" и "Искр". На них заводились реальные заказы, поэтому необходимо вычислить их и переименовать на правильное название — Iskra Bar.

Объявим функцию `replace_restaurant()`, которая заменяет неправильное название, а потом проверяет успешность выполнения замены подсчетом (если замена прошла успешно — результат подсчета должен быть 0).

In [102]:
bo_data['restaraunt'].unique() # выявляем, могут ли быть еще неправильные названия, помимо тех, что описаны в задании
def replace_restaraunt(default_name, replace_name):
    bo_data['restaraunt'].replace(default_name, replace_name, inplace=True)

Передаем функции названия 'Искра' и 'Искр'.

In [103]:
replace_restaraunt('Искра', 'Iskra Bar')

In [106]:
replace_restaraunt('Искр', 'Iskra Bar')

### Выводы

В этом блоке мы:
* Убрали лишние пробелы с названий столбцов, а так же поменяли наименования на английский язык, чтобы было проще получать значения
* Так же мы обозначили с помощью марки *admin* тестовые заказы
* Избавились от дубликатов, во избежании искажения статистики
* Исправили неккоректные заказы на ресторан *'Iskra Bar'*

## Распределение заказов в течение недели

Считаем, на какие кафе приходится больше всего заказов.

In [227]:
bo_data.groupby('restaraunt').count().sort_values(by = 'userID', ascending = False)['userID']

restaraunt
Iskra Bar     287
Там Ем        200
Бар Сибирь    159
Северия       107
Крафтейнер     78
Коворкафе      66
Азия           62
admin          41
Name: userID, dtype: int64

Группируем по `weekday` и считаем, на какие дни приходится больше всего заказов.

In [225]:
bo_data.groupby('weekday').count().sort_values(by = 'userID', ascending = False)['userID']

weekday
Tuesday      251
Wednesday    242
Thursday     180
Monday       141
Friday       110
Saturday      52
Sunday        24
Name: userID, dtype: int64

Объявляем функцию, которая возвращает количество заказов для ресторана в определенный день.

In [190]:
def amount_orders(restaraunt, weekday):
    result = bo_data[(bo_data['restaraunt'] == restaraunt) & (bo_data['weekday'] == weekday)]
    return len(result)

Передадим ей заведения Iskra Bar и Там Ем — два флагманских места, по которым просили результаты прежде всего — и понедельник, среду и пятницу в качестве дня недели.

In [205]:
print(f''' ORDERS:
Iskra Bar in Monday - {amount_orders('Iskra Bar', 'Monday')} 
Iskra Bar in Wednesday - {amount_orders('Iskra Bar', 'Wednesday')}
Iskra Bar in Friday - {amount_orders('Iskra Bar', 'Friday')}
Taм Ем in Monday - {amount_orders('Там Ем', 'Monday')}
Taм Ем in Wednesday - {amount_orders('Там Ем', 'Wednesday')}
Taм Ем in Friday - {amount_orders('Там Ем', 'Friday')}''')

 ORDERS:
Iskra Bar in Monday - 48 
Iskra Bar in Wednesday - 53
Iskra Bar in Friday - 29
Taм Ем in Monday - 25
Taм Ем in Wednesday - 52
Taм Ем in Friday - 24


Сведем результат в таблицу.

In [224]:
rows = ['Iskra Bar', 'Там Ем']
columns = ['Monday', 'Wednesday', 'Friday']
data = [[48,53,29], [25,52,24]]
df_for_task = pd.DataFrame(data, index=rows, columns=columns)
df_for_task

,Monday,Wednesday,Friday
Iskra Bar,48,53,29
Там Ем,25,52,24


### Выводы

В этой части мы можем наблюдать:
* Большая часть резервов в вторник, так же на втором месте распологается среда, в другие дни бронируют не так активно
* Iskra Bar является лидером резервов
* так же мы получили данные по 'флагманским' ресторанам: сколько резервов приходится на понедельник, среду и пятницу

## Распределение заказов в течение дня

Сравним количества заказов, сделанных в обеденное и вечернее время — с 12 до 16 и с 16 до 22. Для этого объявим функцию, которая принимает данные и 2 аргумента времени.

In [379]:
def amount_orders_in_time(dataset, from_time, to_time): 
    return dataset[(dataset['ordered'] >= from_time)&(dataset['ordered'] <= to_time)].count()[0]
    

Передадим промежуток с 12:00 и 16:00

In [380]:
amount_orders_in_time(bo_data, '12:00', '16:00')

588

И с 16:00 до 22:00

In [381]:
amount_orders_in_time(bo_data, '16:00', '22:00')

338

### Выводы

Мы выяснили, в какой интервал приходит больше резервов - оказалось, это дневное время



## Типичное количество человек за столиком

Выясним, для какого количества человек обычно заказывают столик в разное время на примере Iskra Bar и Там Ем. Для этого напишем функцию, которая принимает данные, название ресторана, 2 времени и считает моду.

In [392]:
def typical_amount_guests(dataset, restaraunt, from_time, to_time):
     return dataset[(dataset['restaraunt'] == restaraunt)
                    &(dataset['ordered'] >= from_time)
                    &(dataset['ordered'] <= to_time)]['guests'].mode()[0]

Проверим функцию на двух ресторанах, а также дневном и вечернем промежутке времени.

In [393]:
typical_amount_guests(bo_data, 'Iskra Bar', '10:00', '16:00')

2

In [385]:
typical_amount_guests(bo_data, 'Iskra Bar', '16:00', '24:00')

1

In [386]:
typical_amount_guests(bo_data, 'Там Ем', '10:00', '16:00')

2

In [387]:
typical_amount_guests(bo_data, 'Там Ем', '16:00', '24:00')

1

### Выводы

Используя данные двух, уже известных нам, "флагманов", мы выяснили, что чаще в вечернее время стол бронируется на одного, в дневное - на двух

## Общие выводы

Используя полученные данные, можем сделать вывод:
* Iskra Bar и ресторан 'Там Ем' являются лидерами по бронированию столов через наше приложение
* В обеденное время(до вечернего сервиса) резервов приходит больше, а так же в это время рестораны хотят посетить бОльшее количество человек
* В выходные дни приложении пользуется спросом намного меньше, нежели в будни(особенно во вторник и среду)
